Cluster the orange ball using Single Guassian

In [1]:
# import packages
import cv2
import os
import numpy as np

In [5]:
def cal_mean_cov(img):
    l, w, h = img.shape
    mean = [[np.sum(img[:,:,0])/(l*w)],[np.sum(img[:,:,1])/(l*w)],[np.sum(img[:,:,2])/(l*w)]]
    cov = np.zeros((3,3),)
    for width in range(len(img[:,0,0])):
        for length in range(len(img[0,:,0])):
            RGB_value = [[img[width][length][0]],[img[width][length][1]],[img[width][length][2]]]
            #print("RGB value \n", RGB_value)
            cov = cov + (np.asmatrix(RGB_value) - np.asmatrix(mean))@(np.asmatrix(RGB_value) - np.asmatrix(mean)).T
            # print((np.asmatrix(RGB_value) - np.asmatrix(mean))@(np.asmatrix(RGB_value) - np.asmatrix(mean)).T)
            # print("cov", cov)
    cov = cov/(l*w)
    return mean,cov

In [7]:
# load data
dir = "train_images" # path to the train image dataset
# User defined threshold
tau = 2

for img_name in os.listdir(dir):
    img = cv2.imread(os.path.join(dir, img_name))
    mean, cov = cal_mean_cov(img)
    print("cov: \n", cov)
    break

cov: 
 [[2781.61681836 2878.44650827 2934.94459974]
 [2878.44650827 3007.86237752 3083.16294443]
 [2934.94459974 3083.16294443 3227.16865765]]
